In [0]:
%python
# %sql
# %md
# %run ./Includes/Setup
# %fs ls 'databricks-datasets' # dbutils.fs.ls('databricks-datasets')
# dbutils.help() 
# dbutils.fs.help()

In [0]:
-- USE CATALOG hive_metastore;
-- drop table employee
create table if not exists employees (
  id int,
  name string,
  salary double)
  -- using delta  # default table type

In [0]:
INSERT INTO employees
VALUES 
  (1, "Adam", 3500.0),
  (2, "Sarah", 4020.5);

INSERT INTO employees
VALUES
  (3, "John", 2999.3),
  (4, "Thomas", 4000.3);

INSERT INTO employees
VALUES
  (5, "Anna", 2500.0);

INSERT INTO employees
VALUES
  (6, "Kim", 6200.3)

In [0]:
select * from employees;

In [0]:
describe detail employees;

In [0]:
%fs ls 'dbfs:/user/hive/warehouse/employees'

In [0]:
UPDATE employees 
SET salary = salary + 100
WHERE name LIKE "A%"

-- COMMAND ----------



In [0]:
SELECT * FROM employees

In [0]:
%fs ls 'dbfs:/user/hive/warehouse/employees'

In [0]:
DESCRIBE DETAIL employees

In [0]:
DESCRIBE HISTORY employees

In [0]:
 %fs ls 'dbfs:/user/hive/warehouse/employees/_delta_log'

In [0]:
--  latest file
%fs head 'dbfs:/user/hive/warehouse/employees/_delta_log/00000000000000000005.json'